In [ ]:
import numpy as np
import pandas as pd
import re
import string
import math

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test (1).csv to test (1).csv
Saving training.csv to training.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['training.csv']))
df

,id,text,label
0,1,i didnt feel humiliated,0
1,2,i can go from feeling so hopeless to so damned...,0
2,3,im grabbing a minute to post i feel greedy wrong,3
3,4,i am ever feeling nostalgic about the fireplac...,2
4,5,i am feeling grouchy,3
...,...,...,...
17995,17996,im having ssa examination tomorrow in the morn...,0
17996,17997,i constantly worry about their fight against n...,1
17997,17998,i feel its important to share this info for th...,1
17998,17999,i truly feel that if you are passionate enough...,1


# Removing all the contractions. Ex- didn't, wasn't etc.

In [ ]:
!pip install contractions
import contractions

In [ ]:
df['text'][0]=contractions.fix(df['text'][0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
for i in df['text']:
  i=contractions.fix(i)

In [ ]:
df["text"] = df["text"].astype(str)

# Removing stopwords

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('im')
stop_words.add('ive')
stop_words.add('feel')
stop_words.add('feeling')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
df['text'] = df['text'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df

,id,text,label
0,1,didnt humiliated,0
1,2,go hopeless damned hopeful around someone care...,0
2,3,grabbing minute post greedy wrong,3
3,4,ever nostalgic fireplace know still property,2
4,5,grouchy,3
...,...,...,...
17995,17996,ssa examination tomorrow morning quite well pr...,0
17996,17997,constantly worry fight nature push limits inne...,1
17997,17998,important share info experience thing,1
17998,17999,truly passionate enough something stay true su...,1


# Lemmatization

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
df["text"] = df["text"].apply(lambda text: lemmatize_words(text))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df

,id,text,label
0,1,didnt humiliated,0
1,2,go hopeless damned hopeful around someone care...,0
2,3,grabbing minute post greedy wrong,3
3,4,ever nostalgic fireplace know still property,2
4,5,grouchy,3
...,...,...,...
17995,17996,ssa examination tomorrow morning quite well pr...,0
17996,17997,constantly worry fight nature push limit inner...,1
17997,17998,important share info experience thing,1
17998,17999,truly passionate enough something stay true su...,1


# Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVect = TfidfVectorizer()
TfidfVect.fit(df['text'])
x = df['text']
y = df['label']
x_vector = TfidfVect.transform(x)

In [ ]:
x_vector

<18000x14293 sparse matrix of type '<class 'numpy.float64'>'
	with 141613 stored elements in Compressed Sparse Row format>

In [ ]:
x_vector.shape

(18000, 14293)

In [ ]:
y.shape

(18000,)

# Model Building

In [ ]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()
model.fit(x_vector, y)

MultinomialNB()

In [ ]:
from sklearn.naive_bayes import ComplementNB
model2=ComplementNB()
model2.fit(x_vector,y)

ComplementNB()

####Complement NB worked better in building the model.

# Model Testing

In [ ]:
df_test=pd.read_csv(io.BytesIO(uploaded['test (1).csv']))
df_test

,id,text
0,1,im feeling rather rotten so im not very ambiti...
1,2,im updating my blog because i feel shitty
2,3,i never make her separate from me because i do...
3,4,i left with my bouquet of red and yellow tulip...
4,5,i was feeling a little vain when i did this one
...,...,...
1995,1996,i just keep feeling like someone is being unki...
1996,1997,im feeling a little cranky negative after this...
1997,1998,i feel that i am useful to my people and that ...
1998,1999,im feeling more comfortable with derby i feel ...


In [ ]:
for i in df_test['text']:
  i=contractions.fix(i)

In [ ]:
df['text'] = df['text'].apply(lambda x: remove_stopwords(x))

In [ ]:
df_test["text"] = df_test["text"].apply(lambda text: lemmatize_words(text))

In [ ]:
pred_vector_ = TfidfVect.transform(df_test['text'])

In [ ]:
label = model2.predict(pred_vector_)
label

array([0, 0, 0, ..., 1, 1, 4])

In [ ]:
df_test['label'] = label.tolist()

In [ ]:
df_test.head(20)

,id,text,label
0,1,im feeling rather rotten so im not very ambiti...,0
1,2,im updating my blog because i feel shitty,0
2,3,i never make her separate from me because i do...,0
3,4,i left with my bouquet of red and yellow tulip...,1
4,5,i wa feeling a little vain when i did this one,0
5,6,i cant walk into a shop anywhere where i do no...,4
6,7,i felt anger when at the end of a telephone call,1
7,8,i explain why i clung to a relationship with a...,1
8,9,i like to have the same breathless feeling a a...,1
9,10,i jest i feel grumpy tired and pre menstrual w...,3


In [ ]:
df_test=df_test.drop(columns=['text'])
df_test

,id,label
0,1,0
1,2,0
2,3,0
3,4,1
4,5,0
...,...,...
1995,1996,3
1996,1997,3
1997,1998,1
1998,1999,1


In [ ]:
df_test.to_csv("sample_submission.csv",index=False)

In [ ]:
from google.colab import files
files.download("sample_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df1=pd.read_csv("sample_submission.csv")